## Import Libraries

In [ ]:
# Import Libraries

from arcgis.gis import GIS
import arcgis
import rasterio as rast
from rasterio.control import GroundControlPoint as gc
from rasterio.plot import show
import os
import pandas as pd
import json

# Initialize Licensing 

gis = GIS("pro")

## Define Functions

### Geocoding Function

In [ ]:
def geocode_address(address_list):
    
    for address in address_list:
        address['geo_y'] = []
        address['geo_x'] = []
        address['score'] = []

    for address in address_list:
        geocode_result = arcgis.geocoding.geocode(address=address['address'] + ", Waterloo")
        
        # Extract geocode information
        geocode_score = geocode_result[0].get('score', None)
        geocode_y = geocode_result[0]['location'].get('y', None)
        geocode_x = geocode_result[0]['location'].get('x', None)
        
        # Append geocode values to the respective dictionary
        address['geo_y'].append(geocode_y)
        address['geo_x'].append(geocode_x)
        address['score'].append(geocode_score)
          
    return address_list

### Filter Scoring Function

In [ ]:
def filter_score(geocoded_scores):
    filtered_list = [row for row in geocoded_scores if row['score'][0] >= 98]
    
    return filtered_list

### Georeferencing Function

In [ ]:
def georeference_image(image_path, output_path, geocoded_addresses):
    
    # Access unreferenced raster
    unRefRaster = rast.open(image_path)
    
    # Define output path
    refRaster = output_path

    # Empty list for control points
    point_list = []
    
    for item in geocoded_addresses:
        point = gc(item['y'], item['x'], item['geo_x'][0], item['geo_y'][0])
        point_list.append(point)
        
    transform = rast.transform.from_gcps(point_list)
    
    with rast.open(refRaster, mode="w", 
               driver="GTiff", 
               width=unRefRaster.read(1).shape[0],
               height=unRefRaster.read(1).shape[1],
               count=3,
               dtype=unRefRaster.read(1).dtype,
               crs=rast.crs.CRS.from_epsg(4326),
               transform = transform
               ) as dst:
        dst.write(unRefRaster.read(1), 1)
        dst.write(unRefRaster.read(2), 2)
        dst.write(unRefRaster.read(3), 3)
        
    geoRaster = rast.open(refRaster)
    show(geoRaster)
        
    print("Raster Transformation Complete")

### Renaming File Function

In [ ]:

def rename_file(file_name):
    base_name, extension = os.path.splitext(file_name)
    if extension.lower() == ".jpg":
        new_file_path = base_name + ".tif"
        return new_file_path
    else:
        return file_path

### Reverse Transformation Function

In [ ]:
def pixel_coordinates(image_path, geocoded_address):
        
        # Define new properties for coordinates corresponding to pixels
        for item in geocoded_address:
            item['pixel_y'] = []
            item['pixel_x'] = []
            
        with rasterio.open(image_path) as src:
            raster_transform = src.transform
            
        for item in geocoded_address:
            pixel_x = item['x']
            pixel_y = item['y']
            
            x,y = rasterio.transform.xy(transform = raster_transform,
                                        rows = pixel_y,
                                        cols = pixel_x)
            
            item['pixel_y'].append(y)
            item['pixel_x'].append(x)
            
        return geocoded_address

### Haversine Calculation Function

In [ ]:
def calculate_haversine(geocoded_address):
    
    for item in geocoded_address:
        item['haversine_meters'] = []
        
        pixel_coords = (item['pixel_y'][0], item['pixel_x'][0])
        geo_coords = (item['geo_y'][0], item['geo_x'][0])
        
        distance = haversine(pixel_coords, geo_coords)
        distance = distance * 1000
        
        item['haversine_meters'].append(distance)
        
    return geocoded_address
  
    

## Set Directories 

### Set Paths

In [ ]:
# Image paths and Work Directories

images_path =  "Testing_Clips"
low_noise_path = os.path.join(images_path, "Low_Noise")
result_dir = os.path.join(images_path, "result")
result_out_dir = os.path.join(images_path, "result_out")

os.chdir(result_dir)
print("Current Working Directory:", os.getcwd())

In [ ]:
# Image and Feature File Names

image_name = "WaterlooJuly1892_3.jpg"
image_name_base, image_name_ext =  os.path.splitext(image_name)
ref_image_name = "referenced_" + rename_file(image_name)

feature_list = open(image_name_base + ".txt", "r")
feature_list = feature_list.read()
feature_list = json.loads(feature_list)

In [ ]:
feature_list

### Geocode The Address

In [ ]:
geocode_results = geocode_address(address_list = feature_list)

In [ ]:
filtered_results = filter_score(geocoded_scores=geocode_results)

### Transform the Raster

In [ ]:
georeference_image(image_path = os.path.join(os.getcwd(), image_name), 
                   geocoded_addresses=filtered_results, 
                   output_path= os.path.join(result_out_dir, ref_image_name))

### Reverse Transform

In [ ]:
pixelcoord_results = pixel_coordinates(image_path = os.path.join(result_out_dir, ref_image_name), 
                                       geocoded_address=filtered_results)

### Calculate Haversine Distances

In [ ]:
haversine_results = calculate_haversine(geocoded_address=pixelcoord_results)

### Convert to Data Frame and Export

In [ ]:
df = pd.DataFrame(haversine_results)
df

In [ ]:
df.to_csv(image_name + ".csv")

In [ ]:
geocode_results